In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from scipy import integrate
import math

In [ ]:
variables = locals()
tickers = ['VIX','AGG','DBC','VTI']
def load_data(tickers):
    for ticker in tickers:
        variables["{0}".format(ticker)] = pd.read_excel('data_'+ticker+'.xlsx')
def fuse_data(tickers):
    appended_data = [] #list to store data 
    for ticker in tickers:
        appended_data.append(variables[ticker]['Close'])
    variables["df"] = pd.DataFrame(appended_data).T
    variables["df"].columns = tickers
    variables["df"] = variables["df"].dropna()
def sharpe_calculation(tickers,window):
    for ticker in tickers:
        df[ticker+"_return"] = (df[ticker]/df[ticker].shift(window) -1)
        df[ticker+"_std"] = df[ticker].pct_change().rolling(window).std()
        df[ticker+"_sharpe"] = df[ticker+"_return"]/df[ticker+"_std"]
def allocation_calculation(tickers,weights,window):
    lst = []
    for ticker in tickers:
        lst.append(ticker+'_return')
    for key in weights:
        df[key+'_return'] = (df[lst] * weights[key]).sum(axis=1)
        df[key+'_std'] =  df[key+'_return'].rolling(window).std()
        df[key+'_sharpe'] = df[key+'_return']/df[key+'_std']
        df[key+'_downsidedev'] = math.nan
        for x in range(window,len(df[key+'_return'])):
            loockback_returns = df[key+'_return'][x-window:x]
            negative_lookback_returns = loockback_returns.where(loockback_returns < 0).dropna()
            df[key+'_downsidedev'].loc[x] = negative_lookback_returns.std()
        df[key+'_sortino'] =  df[key+"_return"]/df[key+"_downsidedev"]

In [3]:
load_data(tickers)
fuse_data(tickers)

In [45]:
sharpe_calculation(tickers,50)

In [57]:
weights = {'allocation1' : [0.25,0.25,0.25,0.25],'allocation2' : [0.5,0.1,0.2,0.2], 'allocation3' : [0.1,0.5,0.2,0.2], 'allocation4' : [0.4,0.4,0.1,0.1]}

In [170]:
allocation_calculation(tickers,weights,50)